# Phase 2: Train the single models

In [1]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
set_session(tf.Session(config=config))

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS = 100000
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 30
OUT_SIZE = 1

## Load and prepare the data

In [3]:
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains, tests, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.471 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 30) (320552, 30)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [3]:
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains, tests, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.461 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 30) (320552, 30)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [4]:
from keras.utils import to_categorical
from sklearn.utils import class_weight
labels = to_categorical(labels)

## Prepare word embedding

In [5]:
print("Embeddings")
print(os.listdir("../data/wordvec"))

Embeddings
['.gitkeep', 'fasttext-50-win3.vec', 'sgns.merge.bigram', 'Tencent_AILab_ChineseEmbedding.txt', 'zh-wordvec-50-cbow-windowsize50.vec', 'zh-wordvec-50-skipgram-windowsize7.vec']


In [6]:
data_loader = DataLoader()
sgns_bigram_embedding = data_loader.load_embedding('../data/wordvec/sgns.merge.bigram')
tencent_ai_embedding = data_loader.load_embedding('../data/wordvec/Tencent_AILab_ChineseEmbedding.txt')

Total 1284313 word vectors.
Err on  ['中共中央', '国务院关于完善产权保护制度依法保护产权的意见']
Err on  ['杨', '光']
Err on  ['王', '琪']
Err on  ['食品安全国家标准', '食品添加剂使用标准']
Err on  ['共担时代责任', '共促全球发展']
Err on  ['三藏不忘本', '四圣试禅心']
Err on  ['坚持开放包容', '推动联动增长']
Err on  ['陷虎穴金星解厄', '双叉岭伯钦留僧']
Err on  ['贾夫人仙逝扬州城', '冷子兴演说荣国府']
Err on  ['食品安全国家标准', '食品中污染物限量']
Err on  ['尸魔三戏唐三藏', '圣僧恨逐美猴王']
Err on  ['情乱性从因爱欲', '神昏心动遇魔头']
Err on  ['机动车类型', '术语和定义']
Err on  ['我', '末代工农兵学员']
Err on  ['财政部', '国家税务总局关于非货币性资产投资企业所得税政策问题的通知']
Err on  ['弘扬“红船精神”', '走在时代前列']
Err on  ['陈光蕊赴任逢灾', '江流僧复仇报本']
Err on  ['蛇盘山诸神暗佑', '鹰愁涧意马收缰']
Err on  ['深化伙伴关系', '增强发展动力']
Err on  ['猪八戒义激猴王', '孙行者智降妖怪']
Err on  ['九九数完魔灭尽', '三三行满道归根']
Total 8824309 word vectors.


In [7]:
def build_embedding_matrix(embeddings_index, embedding_size, nb_words=NB_WORDS, word_index=data_transformer.tokenizer.word_index,):
    #nb_words = min(nb_words, len(embeddings_index))
    embedding_matrix = np.random.rand(nb_words, embedding_size)
    word_index = data_transformer.tokenizer.word_index
    null_words = open('null-word.txt', 'w', encoding='utf-8')
    null_ctr = 0
    for word, i in word_index.items():
        if i >= nb_words:
            null_words.write(word + '\n')
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            print(word)
            null_ctr += 1
            null_words.write(word + '\n')
    print('Null word embeddings: %d' % null_ctr)
    return embedding_matrix

In [8]:
sgns_bigram_matrix = build_embedding_matrix(sgns_bigram_embedding, embedding_size=300)
tencent_ai_matrix = build_embedding_matrix(tencent_ai_embedding, embedding_size=200)

两项
一款
这是
过大年
多大
吃秀
一天
这会灯
 
爱自
楠哥
烯碳
a股
第一股
同食
信要
多个
三年
这一
海试
喻言吐槽
扎心
会变
上吐槽
喻言狂
或成
ceodiss
一分钟
常吃
竟能
群助
第三
赌高冰
赌出
超快
一枚
手麻
天去
孕妈
屏有
mix3
这枚
1000vs10
借水
十年
上万元
找下
市偷
此国
加它
三分钟
黑如
组图
霍金的
亿年
几片
养狗场
几千只
一抹
三天
十天
为市
镇有
10w
最严
万左右
日系
三席
最损
物可
条线
人能
群会
北三县
破万
燃脂
几年
一种
蒸出
号线
一分
天吃
天瘦
被切
一首
吃辣条
辣条
一百枚
会值
一万元
一层
几种
一把
突都
包难消
一杯
艾草
给治好
数十万
几个
好几个
治一好
稳降
请存
半个
狂降
速存
欲生
日起
限重
销分
被查
分罚
新交规
惊现
放车
日新规
将会
考驾照
要持
双证
消分
飙戏
车购
无糖
第一次
想不长
最该
有福
生吃
长出
两个
包块
开喷战
我生
男宝
虐童
两证
要罚
审新规
卖不动
h6
一笔
可领
120km
将成
首条
四驱带
一看
比哈弗
万要
几量
没差
级车
美系
逼格
加分制
曝已
颈纹
美过
因丑
一张
5g
投给
25419
怀男
万公里
最脏
七天
十斤
瘦出
半年
一个月
八斤
瘦到
天减
减腰
天让
一条
玩太多
狗伤
不起
三十出头
这件
奶爸
只降
家有
快存
两周
肥要
養顏茶
吃瓜
性侵
辣妈
生完
暴瘦
狂瘦
甩成
这几物
一副
这四物
秘制
仨月
几味
老款
配全
时四驱
这才
玉娆
牛孩
吹大
撒药治
换个
拿奖
三千年
抹平
两种
消脂
宝妈
软肥肚
两物
几款
万拍
这台
一辆
达人教
能当
gl550
去味剂
已变
选股法
多只
日有
冯提
多万
s400l
懂车
一碗
甩腹
几口
排宿
刮光
上百万
一次
一声
岁萝莉
晚清
八年
170508
撩妹
171008
171110
首个
煅荷
物泡
一泡
几天
一场
半月
已花
抖音
妖股
亿封板
下周
连板
墓主
神押题
三味
快学
熬点
补脾
三物
18888888888
万买台
a6l
cs95
直降
男神
五年
生神
泪洒
十万分之一
还育
仍似
千元
交了
从根
三种
再虐
一班
抑癌
四点
四次
这国
快成
第二个
乐视
亿

浙卫
群嘲
袁立开
怒批
条微
博手
浙卫发
只留
袁立手
要凉
遭众
思谈
陈立农
曝单
开录
文千玺
靳东景
除易
孕肚
赵丽颖为
养狗
空天猎
围有雷
之责
双男主
助演
晒出
启月
会换
要弃剧
还强
一万倍
链热
因不敌
只爱
这剧
女主竟
嘴下
演杀
男主霍
诗来
一出
饰子
女主小骨
李沁来
糖宝
演小骨
版小骨
非鹿晗
由迪丽
之大
当不了
房差
不骂
爱豆们
之恋
两人会
若干年
秘客
后星爷
七季
第九集
刀女
群杀
行尸
迅音
退剧
陈赫终
最尬
范丞丞
男患
第一站
男六
细思极
川普当
真该
冲得
打一星
评一星
圣贤书
唐嫣任
e3
二号
想弃剧
演女
加戏
马小翠近
一女
家驹
百分之三十五
添新规
fm889
180509
一群
三口
第一家
违停
下道
谈竟
照谣
出户
环乌
养肝护
再乱
一堆
惊出
稳糖
五块
亏惨
两千年
江西人
喝才
数名
只卖
少吃肉
不疼
穿涼鞋
買點
貨塗塗
腳丫恢
天检荐
无癌
安娃
多会
遭抖音
为空
小编
币圈
币安
带膏
三组
一箭
竖纹
辉凉
六款
妻且
提莫
履蛋收
更多人
撷珍
发大招
美女秀
能怪
瘦腿术
白頭
有解
八個
食療
沒幾天
回來
百日
疯转
知食
寿字币
广等
纤营
撒药灭
bp5
三名
爆火
网评
跌出
老匠
老鸣
一瓣
越香越
闻着
人快
活虫
迅解
远成
一幅
静图
图在动
贷等
网贷
拒贷
闲闻
连跌
54169
吃油
需退
翔性
前名
指唐德
李晨补

不洗
穿会
喜大普奔
增甜剂
后罗
红毯秀
转粉
赵丽颖称
六类
你生
六人
暴恐
城事
此招
必红
含碱
完电不拔
三方
哪般
畅姐
最缺
福布
不图名
必有
诚寻大
多辆
希沃
4g
一毒
美老
就会变
二个
一两个
天必
既护
吸粉
人带
太累
千万元
可治好
拾文
一栋
教它
省下
前配
一擦
小秘方
九条
胡了
二条
四十岁
瘦到斤
十公斤
七道
一治
天即
三周
速瘦
人真会
美哭
尖圆
滚一滚
一点点
一九五三年
一九五
三版
五款
第二次
一个亿
一仗
发嫂
六年
泪别
亲述
曝买
吃活
线乱
形散
三遍
一篇
一個
打麻將
能换
这到
几十倍
纸氏
十万倍
全仓
此股
看怀
还贵
已孕
一剧
5t
成热词
咖总
10gb
4k
一勺
十几年
十副药
十几种
透白
一卷
太逗
四天
却治好
五步
因

别错
几所
学渣成
关晓彤疑
学鲁能
超换个
凭此
里生
你速
减衣
只靠
伤脑
泡个
吃此
好长
多转
跳错
高不高
聪不
被拉到
却装
长不高该
长高靠
妈别
河沥溪
吴忠至
两女
市住
队规
私接
拒游
坑村
曝微
博说
六公里
肾如
三高稳
身壮似
它堪
人不爱
放锅里
一蒸
保肝护
害草
超爱
减脂王
no1
喝能
却常
个通
更粉
常吃固
茶中
中之果
之宗
它配
林洗
多房
不拿钱
万亿度
稳进
一队
可成
德帅
四侠
几成
若去
彭胡弯
搞基
越用
越卡
锁屏
干过
要会
七百万
人必
证上
焚屋
遭人割
四六级
七口
百丈
有多准
便知
凤祥银
比巴图
图可要
几千几万
马蓉求
疑向
我定
博放狠话
之苦
并放话
会输
宋喆放话
马蓉称
其毁
宋喆疑
豪掷
疑涉
会判
最多会判
可宝强
涉此
马蓉禁
重查
微博求
脸求
心大
宋喆要
他气
程野大
巨火
重疾
太大要
活一活
认真听讲
首秀
f4
曝暂
小撒
宋爆
小s
又力
宋祖得
几首
想红想
坠桥
算出去
可长
骂遍
单替
生不动
舞力
揭施
一公
称施
分小
再伤
全倒
virgil
abloh
前国足
美加墨
一站
三检
乐遭
而致
优畅
街夏联
宝奶
有接
易使
一员
流黄
胖越
重越
第八天
总吃
断夜奶
粘人
一喝立
越治越
立退
一百倍
澡后
酸爽
全不
溶豆
选存
四宗
脑后
别因
咬出
期吃
磨娘
越难
被裁
身睡
x7
宝马新
案刚
案大
4l
比宝骏
我开
多平
取卵
似港
匪片
疑现
放瓶水
拿水
黑拳
捕到
光人
人出
爆怀
实拍用
整颗
几十只
太辣眼
实拍超
棍状
杀后
数十斤
秒速
开秀
韩姨爆
会恋
诗双
宣宝
芳儿
椒园
多座
喷一
宪哥
热足
一万只
活多大
绿植能
七只
两把
半患
从何
种能
吸雾
再脏
杨紫林
吸废排
第二名
很卡
边放上
全给
宜忌
扔教
套上
难擦
发钝
如新
牛人教
需放点
丢点
数十倍
土壳
一刷
称能
喂活
injuv
莹久
招清宿
众鑫活
羊馆
盛悦府
白白嫩
亿成
24h
万招
养着
马踏
贩婴案
补氧神水
何孟怀
买艘
曾酒
美中印
最逗
好防
王凯版
雪姨有
耍个
对飙
五三年
王思聪家
被象
象夫
象尾
追人
再存
赵丽颖何
董洁王
进户
不愿用
盛一伦小妾
将准
泡入
家長
送豪车
小七暴
这颜值
王俊凯学
五百公

马基夫
勇骑
能夺
g5
大减
莫雷点
神塔要
亿将
和保罗
首胜
同吃易
实指
親測
一具
胃及
罗家英
增晰酷
炒币
放太多油
将现
股神大
买新锅
别盖
论看
真炸
还大涨
增近
点盘
颗烟
什麼
雅瑪人
預言
有三種
希伯來語
可亏
但用
抽才
烟里
纸丝
一排
闹崩
郭德纲面
后别
全乱
继林丹
热拉提
一个班
日漫
改看
焦曼婷
恒大准
焰楠
别放
香弹
粘壳
王渣
提王
对马蓉
比马蓉
强太多
更毒
熏醋防
补觉
几期
觉补
不长个
吸六包
避堵
忧难
侠会
爆帅
萌照
萌翻
太仓促
有多累
爆李
爆照
别太拼
爆看
爆娱
带货
糊得
爆马蓉妈
帮马蓉
肉会
爱吃西
爱吃鹅肝者
爱大爱
爱尚轻
凉要
吃狗娘
水与身
爱芯
两席
爱酒
爱黛儿
我成
寻子
破壶
四弟
三权
按置
父皇要
点想长
好眠
吴京包
毕雯珺
遭知乎
越低
撕娜
亿小
招辅警
没块
表贵
牙太黄
宝妈变
脸变
刷鞋
混在
巧加
伤牙
挤太多会
完别
黄别
上满
洗牙强
黑渍
买美白
渐白
太黄
王奶奶
美白仪
牛三犇
牛人用
能堆
混喝
还伤
igf
促癌
喝奶
美白变
暗黄变
连袋
机没造
消癣
简版
妖龙
猛干
有美人
牧院
再聘
员转
业会
或遇
为降
四季度
拉至
万内
男宝要
时遭
亲承
愿回
曝汪峰
会学
亿送
比天
十号
折回来
进雪里
仔列沙
胡可要
狗尿会
六登
喜忌
精会
而刀郎
半狗
生崽
喊人
附超
粮来
装一
七海
证可领
领吧
多分钟
八只
超优德
猎云
新英
安个
太硬
天不烂
传首
猪价
先备
首安茶
卖多
肉能
往家
肉堪
少吃点
猫成
一两天
难有
那烂陀寺
为博
几十人
红沙皮
需一剪
剪后
百分之四十
百分之八十
百分之五十
米须
追美合
王一博美合
王三喜
无湿
王丽坤林
爆林
我特
逆袭成
手残
王源成
他成
护易
男大
再摊
往死里整
已马
问马云
捐后
疑骗
该负
因钱
他选
蒋欣发
微博甜
走虐
同框照
将生
网传王
六小时
同逛
关系暧昧
打鲁能
王如峰
文男
不归路
领家
博谈
宋喆该
强交新
马蓉强
强替
宋喆养
网传宝强
中泪点
强泪
熊乃瑾秀
很正
眷侣
讽其
188c
强前
称若
比马蓉靓
宋喆恐
微博疑
长点
强宝强
宋喆会
宋喆终
终进
宋喆进
强摊
发博称
超马蓉
胜马蓉
熊乃瑾疑
称马蓉
爆宝强
京金联
赵薇版


偏治
大梅沙
越扫
王石陷
比翁帆
胸会
养鹅场
王大哥
八百元
红罐
很扎心
两人用
用泪
枣同
没转
专瘦
黄恒泰
飞防真
打谢娜
谢娜讽
现张杰
a9
128g
两周岁
愿宝
附肚型
规后
否者
天喻
创局
区片
更别
低开
探低
之拥
能养颜
种算
人真
不餐
黑会
陈醋加
抽根烟
某微博
什们
蓉江
无主之地
有前
给霆锋
谢贤立
买首
有楼
没奶
斑多
常吃此
吹霉
十七斤
为大婚
十二斤
通审
四五年
市驾考
园方
波新
和准
起交
额有
这起
化限行
号限行
行新规
卡友圈
五系
第三轮
天弘
没招
变瓜
一美
整月
原子城
九景衢
宝兰客
联试
留放
可防脱
传造
恋再
现又陷
变虚
偷排
百余辆
谢贤疑
八十多岁
谢贤刚
谢贤谈
十六岁
沛纳海
财產归
交个
谢贤想分
吃味传
谢贤称
埋于
曝加
车油
拟运向
指系
行牛
控水
互呛
过高要
茶越
无降配
多市
五象
始作
谣者
蓝鸥
还处
三考
立辨
很乱系
周超快
全了
附超全
已诞下
好孕气
重男
生宝状
数十秒
美羊羊
人吃生
从许
看恋童
网红许
指恋童
踏俩
组间
晨重
第二阶段
辣妈用
练鬼步
速码
天脱
猛瘦
亲荐
可减
日减
甩肚
脂茶
三十九天
每招
姜蜜水
第四天
好几斤
第七天
美拍
甩油
共瘦
莫暴
精脸
大呼求
神舞
极塑
体轻
单腿
特减
第二十三
半根
中圣品
知爽
含屎
没响
推伤
大卧
姜泡
每早
无三高
遭许
晴批
小威威
宋雨洪
对车
政解
越高越
高鑫磊
现神
癌犯
以微博
薛之谦个
周杰互
最靓
四摄
崩人设
坑夫
没睡
安和桥
谦谦爱
业之峰
玩得
王小鸥
真忙
鼓不起
内要
网剧令
几十载
怪孩
返本型
油会
美系车
万近
还享
曝卖
领弹
王一博自
消保委约
alpd
而生
人练
老铁要
里会
粘肠
错得
英签
青微博
无墅
罗曼尼
喝多会
分变
痴汉脸
lms
拖米
karsa
已好
4am
放人
任栋
没说
koro1
带妹
怪用
ziv
查飞单
许知
am4
矿圈
禁挖
f2pool
吴评鑫
转势
删路
反招
比脸
被尬
黑假
其言尽
拍吻
戏用
撒内宁
翻路
辱华
凭颜值
该查
脱库
轩墨
之扣
中毅
况丽任
元股
pmma
科通
芯城
中酒协
灿笑
附选股
无应
或迎
被否
更严
日要
假米
含微
杨颖太像
显矮
一人带
曝假
这大招
安

...

多翅多
尿得
吹多会
三海
网传颍东
树偷
谢旗营
麦明
侵遭
第三度
肾功
张艳家
证全
品蟹
办张
玩一玩
孕检时
那孕妈
需营
肉生
会滑胎
买豪车
再引
争破头
太大意
第三件
种孕妈
骨瘦如材
最养胎
孙像
上六下
团中
邓超怒
闫学晶谈
强林丹
盼夕
信痛
圆版
洞主
克拜峰
位面
脸胖
张翰强
时倒点
13596783427
成实外
再学
卡及
梁伟瑜
梅内塞
一大截
称清
房何
發燒
降溫
五種
爆王鸥
小脏
神床
何炅于
眼痛
用口
趴睡
裤会
种点
扔颗
二十五年
怪树
三郎
树高仅
芦丁有
火菜
跟队
韩餐
瑞思迈
别向
刁然
开借
三高降
可白过
不侵
一波儿
一绿萝花
完一查
藏石
万多块
欲放
马大哥
缅币
一补
五遍

茶威
chx19820524
中太
说收
交吧
溫度
萬億度
立馬產黃金
吞恆星
请关
神将
令莫雷
拒送
前六已
怀德桥吾悦
粉粹机
仙要
共设
受马蓉
撕马蓉
听张
胜多
负少
枣片
天吃出
头用
天如
太多要
匹林同
配阿司
水里系
锁桩
咱不背
淹会
蒋坑
二楼
变票
两人恐
接马蓉
太污
证清
现遭
宋喆后
景甜用
微博艾特
真连
非王
病中
宋喆怒
强终
敌马蓉
第一把
等言伦
谈马蓉
半口
九辰
除防
程野宋
争峰
军会
黄圣衣
曝星爷
葛素珍
人父
妈比
孙俪来
轨才
桌伟要
爆邓
轨江
一燕
同剧
好虐
捧作
那篇
将行
赌上
眼伤
分能
深脑
宁继春
两草
一果
fm1076
复传
别以
撒钱
打不死
药治
阿左旗
六翅鸡
已因
正对
石硫
翻价
后越
此数
因孕妈
总穿
肿成
长不胖
一滚
防刺
六件
柚柚妹
会坑
宝妈图
点不输
出牙期
双待版
超窄
黄橙褐
强讨
所忧
所扰
肉知
肉肉长
总饿
研特膳
美白治
事竟
种暴
腹婆变
四分钟
帅哥美女
周暴
招就够
很痛
版币
五六米
撞断
数个
上实
奈飞
分许
几间
拍剧
早火
不抱
好于
共破
核裂
新核
中天微
商乐鑫
博主刚
马云超
直咒
两百多
粒药
误开
之敌
拼酒
急征
水里配
如脂
乔欣撞
补肽
上值
猛翻
卖套
五千块
胖哥教
睡易
钱制
造因
我领
说领
拆太早
需一土
门引
一长
黏锅
愁教
总粘锅
爱簪
放闪
真狠
王思聪独
十一点
极酸
男卑
王一博刚
爆与富
网传富
常搓
能能
往死里
富氧
带货王
第一轮
弊利
眼用

强光照
h20
可伶
选座
认真思考
之鬼门
红块
其咎
正以
家会
关晓彤生
看怒
胡路区
劳肥
管办
升哥
许晴太作
暗批
和现
爆恋童
转自
献爱王
该出
准不信
可鱼
想演
分卖
已使
蓬出
用非
烯快
争疯
黑稿
侧空
只会涨
台会神
王思聪放话
普制
被性
侵者
做要
维嘉停
录何
走定
话暖心
十几次
坐娜姐
杨迪送
吴昕会
诞子
听林丹
谢菲恋
谢少
提小
谢贤慌
谢贤死
想替
谢贤邀
亿换
形只
影单
亿难
字霆锋
五一回
港办
爸害
出恐
谢贤自
失得
柏芝家
一念间
传喜
前似
完无一
十辆
疑隔空
没尽
竟想花
亿把
酒口
3xl
钙王
e79w32
运死
旧爱大
高管互
供其
照杨
帮甜馨
愿得
实李
想秀
卖人设
带群
迪玛希
曝办
脸亮
太响
给乐视
乐视命
劳福德
君花
赌石大牛
高货
奥拓换
莫湾基
帕克恐
慈善赛
赫子铭称
涵力
李沁女
赵丽颖人
赵丽颖仅
撤有
冯绍峰疑
拍添
赵丽颖冲
你颖宝
张翰俩
赵丽颖长
框演
赵丽颖喜
宣新
高奢
冯绍峰家
装疼
赵丽颖官
宣完
赵丽颖当
何炅笑
爆不雅照
已均
发盐
惹人赞
扣图
前上
有文替
武替
论卧
接剧
混剪
棋帅
赵丽颖现
真恋
修图师
他秀
冯绍峰帅
他帅
陈晓说
谈场
曝未
曝楚
爆楚
乔传用
抱替
撒起
连范爷
系男神
我美
五家
可及
敲迪丽
反笑
一世英名
毁光
卖炸串
已尽
看球球
炫迈
向利哥
小犟
哲家军
帮利哥
小安九入
九局
首麦
家似
曾欲雪藏
立遗
争仪
自比
见利哥
带带
赵武儿
密会迪丽
好成
认马云
快气
曝和马云
七白膏
要叠
国了
后似
韩姨指
乱战出
愿取
越毒
对国足
提力
速干
韬会
鹿含
缓震
我怒
闺蜜学
随流
速卖通
开纯电
灯是
混差
肝毒防
疤膏
亿主播
旺己
旺子
铁元素
取车
未配
不偷
集满
积赞
b2c
新大招
碰维
轻加
艾炙来
干满
而里
人取
个月速
殘疾人
能夠
返厂
美人魚
纸皮
六千元
近加
近半年
一春
大耳雷子
博主称
迅播
多少度
繁事
太毁
价才
没枪
恐伤
还记
亿帮
越富
很受
鱼别
特伤
种假
虽大
稳山
娃恨
伙起
三瓶
扶车尬
用万人
辣竟
这乱
再香
越有
吴京成
男到
太背
事来
点醋
别常
二哈成
这字
别想申
萌到
赵丽颖远
吃似
元币
竟卖
吃助
咖酚
赵丽颖遇
黄西发
陈自瑶同

赵丽颖要
比马蓉强
微速览
鹏疑
老铁们
何炅见
娱姐
关晓彤糊
王思聪米
王俊凯迪丽
说太辣
招洗
手台费
获重判
岁潮
豪命
怒怀
孕妈常
那可要
孕妈生
养颜去
长高个
需不需要
男宝帅
女宝美
条怀
女宝妈
选不对
孕妈别
服就够
邓超要
赵丽颖竟
陈晓任
16620781034
孙杨劳
姚晨办
专将成
学渣成
关晓彤疑
学鲁能
超换个
用得上
被拉到
长不高该
长高靠
吴忠至
拒游
大大的
曝微
三高稳
身壮似
人不爱
保肝护
减脂王
常吃固
遭人割
有多准
凤祥银
图可要
博放狠话
并放话
宋喆放话
马蓉称
宋喆疑
最多会判
可宝强
马蓉禁
微博求
宋喆要
程野大
太大要
曝暂
宋爆
宋祖得
算出去
生不动
揭施
前国足
乐遭
街夏联
一喝立
似港
放瓶水
实拍用
实拍超
韩姨爆
活多大
半患
一去不复返了
杨紫林
吸废排
边放上
厚厚的
扔教
牛人教
见不着
需放点
积极参与
injuv
莹久
招清宿
众鑫活
盛悦府
补氧神水
美中印
王凯版
雪姨有
王思聪家
赵丽颖何
董洁王
不愿用
盛一伦小妾
小七暴
这颜值
王俊凯学
食人蜂
问岳云鹏
竟射出
招太好
驾扣
变程野
文松面
照终
音界里
遭短
真机谍
做安卓
单雷军
乐视存
拼红米
买贵补
直怼
消不掉
发多狠
多小妙
有效率
其滚出
或大降
男不离
驱蟹
这八本
豪高鑫
周琦再
天就治好
昆凌为
坂东永辉
万在华
比瞬秒
传摩拜
北沂堂
评制
三鲁能
杨润达
通血补
湿毒治
时可加
山渣加
编的民
于谦忙
认岳云鹏
于谦成
时冲过
大鑫城
峰菲恋
崔天临
csc8952
3u8633
谈三大
部怒
紧接着
托孟婆
左旗林
显白显
造壹仙
還能
卖员
竟高达
微博放话
句话解
币安买岛
币安在
美白显
帅丕张
翰面
张翰大
迎三大
少一人
愿鲁能
菩洱
配一宝
肝护
可有福
不愿收
里巧放
干不了
胃不痛
从三楼
现扫
交违
库平七钱
恒大来
罗韦世豪
广東
几十个
多大恨
没拉下
吞物门
称可供
莫隐婚
遭石锤
稿齐
应易璐
康震为
救蛇保
几百个
新用式
别慌有
杨紫周
实锤图
秀同款
曝实
杨紫微
张一山恋
杨紫要
被抛弃了
张伦硕带
完候
爸人脉
蒋欣遇
截胡小
局狂
陈晓退
亨离
张杰录
叫炅炅
杰粉
中谢娜
成为事实
还合开
二十个
和好如初
遭霆锋
赶过去
新戏定
亿争
子不当
再多

房菊尼
瘦肚腹
這四種
新用疗
快拿瓣
路虎塔塔
要会用
湿减
皂苷能
补阳強
袁篇
发版别
精发版别
马云问
赫欲
活不了
驻美前
开毫车
中國七大壽星
真成国
人颜值
附币商
养颜不显
更养颜
常吃安
养颜排
既养颜
养颜助
常吃助眠
越美有
人微博
恐不敌
吴京必
抵不上
为国点
种不出
咖开
游梅城
人不嫁
漂拳
赵蕊蕊终
川皇
外媒争
维中超
阿媒爆
某媒称
没干过
雷军排
微博太
阿乌巴
成吐槽
常林回
种人领
遭泼油
李秋平外
信反信
亿小散
大暴击
无数个
俞凌雄点
超坚瑞沃
超赣锋
微封
博聞社
一逆天
子之父
莱口
巨魔成
出蓝贴
极寒来
startfragment
迎特
2018121
示软求
亮亮的
亮哥心
王思聪卓伟
爱嘻哈
探班求
马蓉微
卓尔写
医三人
有钱赚
人不告
糙发
艾尚彩
陷商票
多梦太熬
放太多
购内少
苏索辟
权健亚冠
恒大真
核闹
扎哈维转
权健换帅
权健要
抢人换
成索萨
李霄鹏任
一土帅
阿隆来
有鲁能
李总任
换帅系
上港成
恒大要
杜煜征
津媒欲
宝斗石
scoal
星人怕
临冠路
宋集坞
指不收
害娃
小三假
出道史
小三后
如一人
这小三当
小三史
官博已
热聊引
接甜馨
生咬蛇
宝妈别
引骂战
甩马蓉
卡补审
胖疑
马云送
恒大欲
黄磊首
铁卫吉尔
用纳尔
骚男道
往鲁能
针多针
遭洪爷
恒大真核
人常去
为北飞
曝向
惨触
网红竟
录抖音
没人教
辽足求
飙泪答
殷桃要
雪姨豪
对不上
推遭
很难说
150825
文钱渡
胆码助
说乐视
网传亚冠及
爆乐视
微博致
乐视酷
救乐视
致百人
顾颖琼发
曝贾跃亭
乐视员
张继科生
王丽坤恋
爱宝强
蒋欣自
这四宝
显老像
变年经
咖级视
袁立互
复播成
微博均
袁立勇
天不腐
在此之前
雄安人
时飘窗
完飘窗
皮薄肉
招辨
母梨比
一个包
微博手
来产奶
授破谣
喃再
曝付
成红人
尼美舒
美团涉
一个团
餐员竟
房太难
汪小菲护妻
时宝妈
信酸儿
养颜是
朱之文该
朱之文付
之文于
遭辅警
东峡克麻
河超警
网传东
神碰瓷
仍不信
融冻泥
微博微
称系毒
莫信谣
谢弃
演之恩
杨紫竟
接之恩
马曼玲
潘绫莹
正忙着
鸡主播
德云色家
妹晒
汤爹
脸马蓉
瓜可防
阿信力
仍蒸
很逆天
专吃生
赞井柏然
类加分
车内放
有车族
狗尿伤
加二物
日蓝牌
许鄢
拿卡

似墨别
发再野
如雨落
虽美白
皱显
凶白
斥小三
疑指赖
公媒
李荣浩去
我来养
遇收
佟顺
杨紫问
男神名
草有主
夜互
发照秀
用六字
当谢娜
懂博越
清肠养
时黃瓜別
脂掉
有奇方
湿毒调
秘透
决密
还斗得
mp467
马云爆
马云透
吴京触
马云分
一个千
穷请
马云排
马云财
曝马伊
乔恩遭
微搏求
微博带
讽王
马蓉过
大孕肚
鹿晗发
打马蓉
送宝强
送马蓉
马蓉扎
叫宝强
微博初
买靓房
宋喆称
搜之马蓉
马蓉怒
马蓉时
谢贤八
微博之言
陈坤红
迅怒
那英迫
希疑
希暗
陈晓面
陈晓臭
遭瞎传
希钓出
林丹们
探班白
偏高别
今恋
继惠氏
何孟怀要
港媒称
何孟怀于
吻界
空棘
智穹
人瞬秒
用新锅
列三大
韩红邀
韩红气
餐坏
最可选
首晒孕
批太作
防動脈
手麻治好
需内调
加橘
完可算
喷太
指鹿晗
无真爱
配骁龙
两大癌
脑吃个
rgn0jgq
多大点
晒妈
赵又廷疑
连竿到
小白到
保肝强
字衫
消脂王
太大易
养森
高鑫要
g6602
爆闹
虽劲道
加两宝
剝掉
营尚媒
薛之谦当
不爱何
张凯毅
吃杏肉
王思聪立
鹿晗确
依小三
依继小三
爆黄圣
归家过
黄渤把
黄渤一
洗膏
张一山套
Null word embeddings: 8468


# Trick Features

In [9]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [10]:
rumor_words_list = ['辟谣', '谣言', '谣传', '传谣', '澄清', '真相', '假新闻', '传言', '造谣', '假消息', '不实', '勿传', '假的', '子虚乌有', '诈骗', '骗局', '以讹传讹']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    energy = 0
    for rumor_word in rumor_words_list:
        if rumor_word in text:
            energy = 1
    return energy

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [11]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [12]:
def _build_exact_match_sequences(sent_1, sent_2):
    sent_1_char_set = set(sent_1)
    sent_2_char_set = set(sent_2)
    intersection = sent_1_char_set & sent_2_char_set
    
    sent_1_em = np.zeros_like(sent_1)
    sent_2_em = np.zeros_like(sent_2)

    for i in range(len(sent_1)):
        if sent_1[i] == 0:
            continue
        if sent_1[i] in intersection:
            sent_1_em[i] = 1
    
    for i in range(len(sent_2)):
        if sent_2[i] == 0:
            continue        
        if sent_2[i] in intersection:
            sent_2_em[i] = 1
    
    return sent_1_em, sent_2_em

def build_exact_match_sequences(sents_1, sents_2):
    sents_1_em, sents_2_em = [], []
    for sent_1, sent_2 in zip(sents_1, sents_2):
        sent_1_em, sent_2_em = _build_exact_match_sequences(sent_1, sent_2)
        sents_1_em.append(sent_1_em)
        sents_2_em.append(sent_2_em)
    return np.array(sents_1_em), np.array(sents_2_em)

In [13]:
%%time
trains_1_ems, trains_2_ems = build_exact_match_sequences(trains[0], trains[1])
tests_1_ems, tests_2_ems = build_exact_match_sequences(tests[0], tests[1])

Wall time: 8.77 s


In [14]:
print("Shape of train em", trains_1_ems.shape, trains_2_ems.shape)
print("Shape of test em", tests_1_ems.shape, tests_2_ems.shape)

Shape of train em (320552, 30) (320552, 30)
Shape of test em (80126, 30) (80126, 30)


In [15]:
em_train_features = (trains_1_ems, trains_2_ems)
em_test_features = (tests_1_ems, tests_2_ems)

# Tricks ?

In [16]:
use_tricky = True

if use_tricky:
    trains = (trains[0], trains[1], trick_trains_features)
    tests = (tests[0], tests[1], trick_tests_features)

In [18]:
model_manager = ModelManager()

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs

    def train_folds(self, X, y, fold_count, em_train_features, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        X1, X2, features, = X
        em1, em2 = em_train_features
        features = features
        weight_val=scale_sample_weight

        fold_size = len(X1) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(X1)

            train_x1 = np.concatenate([X1[:fold_start], X1[fold_end:]])
            train_x2 = np.concatenate([X2[:fold_start], X2[fold_end:]])
            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            
            train_em_1 = np.concatenate([em1[:fold_start], em1[fold_end:]])
            train_em_2 = np.concatenate([em2[:fold_start], em2[fold_end:]])
            
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])
            
            val_x1 = X1[fold_start:fold_end]
            val_x2 = X2[fold_start:fold_end]
            val_features = features[fold_start:fold_end]
            val_em1 = em1[fold_start:fold_end]
            val_em2 = em2[fold_start:fold_end]
            val_y = y[fold_start:fold_end]

            fold_pos = (np.sum(train_y) / len(train_x1))

            train_data = {
                "first_sentences": train_x1,
                "second_sentences": train_x2,
                "mata-features": train_features,
                "first_exact_match": train_em_1,
                "second_exact_match": train_em_2,
            }

            val_data = {
                "first_sentences": val_x1,
                "second_sentences": val_x2,
                "mata-features": val_features,
                "first_exact_match": val_em1,
                "second_exact_match": val_em2,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=weight_val)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        #val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        val_data =  (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         class_weight=class_weight,
                         callbacks=[early_stopping, model_checkpoint],)
        bst_val_score = max(hist.history['val_loss'])
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [20]:
def weighted_accuracy(y_true, y_pred):
    # hard version, 0.5 counts as correct
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # hard
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

In [21]:
def get_dense_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    meta_features_input = Input(shape=(36,), name='mata-features')
    
    
    embedding = Embedding(nb_words, embedding_dim,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.2)(q1_embed)
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.2)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    
    q1_encoded = th(q1_embed,)    
    q2_encoded = th(q2_embed,)
    
    q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)
    q1_encoded = Concatenate()([q2_aligned, q1_encoded])
    q2_encoded = Concatenate()([q1_aligned, q2_encoded])  
    
    cnn_init = Conv1D(42, 1, strides=1, padding='same', activation='relu')
    q1_seq = cnn_init(q1_encoded)
    q2_seq = cnn_init(q2_encoded)
    
    cnns = [Conv1D(42, 3, strides=1, padding='same', activation='relu') for i in range(3)]
    trans = [Conv1D(32, 1, strides=1, padding='same', activation='relu') for i in range(3)]
    
    
    for idx, cnn in enumerate(cnns):
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        q1_encoded = Concatenate()([q1_seq, q2_aligned, q1_encoded])
        q2_encoded = Concatenate()([q2_seq, q1_aligned, q2_encoded])            
        q1_seq = cnn(q1_encoded)
        q2_seq = cnn(q2_encoded)    
    
    
    #capsule_pooling = Capsule(num_capsule=3, dim_capsule=600, routings=2, share_weights=True)
    
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    attn = AttentionWeightedAverage()
    
    
    q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])
    q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])    
    
    
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.5)(h_all)
    #h_all = Highway(activation='relu')(h_all)
    #h_all = Dropout(0.2)(h_all)
    #h_all = Highway(activation='relu')(h_all)    
    h_all = Dense(128, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, meta_features_input], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [22]:
def numpy_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = (y_pred > 0.5).astype(int)
    
    res = np.equal(y_pred, y_true) * weight_mask / np.sum(weight_mask)
    res = np.sum(res)
    return res

# DenseCNN

## TenCent

In [ ]:
fold_count = 8
#embedding_matrix = sgns_bigram_matrix
embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 200

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordTC-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_scale_sample_weights = None
    model_patiences = model_manager.model_patiences[i]
    
    #model_class_weights = {0:100, 1:1.5, 0.9: 3}
    
    def _agent_get_model():
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=1024,
        em_train_features=em_train_features,                                             
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=20)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    score = numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)
    

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      20000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 30, 200)      0           embedding_1[0][0]                
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500


In [ ]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

## SGNS

In [46]:
fold_count = 8
embedding_matrix = sgns_bigram_matrix
#embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 300

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_scale_sample_weights = None
    model_patiences = model_manager.model_patiences[i]
    
    #model_class_weights = {0:100, 1:1.5, 0.9: 3}
    
    def _agent_get_model():
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=1024,
        em_train_features=em_train_features,                                             
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=20)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    score = numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)
    

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 30, 300)      30000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_45 (SpatialDr (None, 30, 300)      0           embedding_34[0][0]               
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 47s 167us/step - loss: 0.7785 - acc: 0.8062 - weighted_accuracy: 0.7607 - val_loss: 0.6431 - val_acc: 0.8494 - val_weighted_accuracy: 0.8084
Epoch 2/500
280483/280483 [==============================] - 42s 151us/step - loss: 0.4264 - acc: 0.8485 - weighted_accuracy: 0.8066 - val_loss: 0.3102 - val_acc: 0.8600 - val_weighted_accuracy: 0.8283
Epoch 3/500
280483/280483 [==============================] - 43s 152us/step - loss: 0.3001 - acc: 0.8650 - weighted_accuracy: 0.8386 - val_loss: 0.3052 - val_acc: 0.8624 - val_weighted_accuracy: 0.8362
Epoch 4/500
280483/280483 [==============================] - 42s 151us/step - loss: 0.2841 - acc: 0.8739 - weighted_accuracy: 0.8499 - val_loss: 0.3058 - val_acc: 0.8618 - val_weighted_accuracy: 0.8364
Epoch 5/500
280483/280483 [==============================] - 42s 151us/step - loss: 0.2715 - acc: 0.8803 - weighted_accuracy: 0.8573 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4324 - acc: 0.8091 - weighted_accuracy: 0.7725 - val_loss: 0.3159 - val_acc: 0.8560 - val_weighted_accuracy: 0.8264
Epoch 2/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.3206 - acc: 0.8536 - weighted_accuracy: 0.8271 - val_loss: 0.3040 - val_acc: 0.8606 - val_weighted_accuracy: 0.8401
Epoch 3/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2972 - acc: 0.8668 - weighted_accuracy: 0.8420 - val_loss: 0.2977 - val_acc: 0.8654 - val_weighted_accuracy: 0.8435
Epoch 4/500
280483/280483 [==============================] - 42s 150us/step - loss: 0.2823 - acc: 0.8745 - weighted_accuracy: 0.8510 - val_loss: 0.2876 - val_acc: 0.8691 - val_weighted_accuracy: 0.8453
Epoch 5/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2702 - acc: 0.8806 - weighted_accuracy: 0.8581 - val_lo

...

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 47s 167us/step - loss: 0.4382 - acc: 0.8089 - weighted_accuracy: 0.7705 - val_loss: 0.3475 - val_acc: 0.8383 - val_weighted_accuracy: 0.8113
Epoch 2/500
280483/280483 [==============================] - 43s 152us/step - loss: 0.3192 - acc: 0.8545 - weighted_accuracy: 0.8279 - val_loss: 0.3338 - val_acc: 0.8455 - val_weighted_accuracy: 0.8280
Epoch 3/500
280483/280483 [==============================] - 43s 153us/step - loss: 0.2953 - acc: 0.8684 - weighted_accuracy: 0.8443 - val_loss: 0.3173 - val_acc: 0.8574 - val_weighted_accuracy: 0.8367
Epoch 4/500
280483/280483 [==============================] - 43s 153us/step - loss: 0.2801 - acc: 0.8761 - weighted_accuracy: 0.8529 - val_loss: 0.3144 - val_acc: 0.8571 - val_weighted_accuracy: 0.8416
Epoch 5/500
280483/280483 [==============================] - 43s 152us/step - loss: 0.2677 - acc: 0.8826 - weighted_accuracy: 0.8607 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 48s 171us/step - loss: 0.7685 - acc: 0.8062 - weighted_accuracy: 0.7609 - val_loss: 0.7026 - val_acc: 0.8274 - val_weighted_accuracy: 0.7866
Epoch 2/500
280483/280483 [==============================] - 43s 153us/step - loss: 0.5337 - acc: 0.8509 - weighted_accuracy: 0.8071 - val_loss: 0.3283 - val_acc: 0.8474 - val_weighted_accuracy: 0.8136
Epoch 3/500
280483/280483 [==============================] - 43s 154us/step - loss: 0.2988 - acc: 0.8657 - weighted_accuracy: 0.8372 - val_loss: 0.3201 - val_acc: 0.8539 - val_weighted_accuracy: 0.8323
Epoch 4/500
280483/280483 [==============================] - 43s 153us/step - loss: 0.2819 - acc: 0.8755 - weighted_accuracy: 0.8502 - val_loss: 0.3173 - val_acc: 0.8541 - val_weighted_accuracy: 0.8297
Epoch 5/500
280483/280483 [==============================] - 43s 153us/step - loss: 0.2697 - acc: 0.8819 - weighted_accuracy: 0.8581 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 49s 174us/step - loss: 0.4362 - acc: 0.8116 - weighted_accuracy: 0.7746 - val_loss: 0.3587 - val_acc: 0.8298 - val_weighted_accuracy: 0.7956
Epoch 2/500
280483/280483 [==============================] - 43s 154us/step - loss: 0.3074 - acc: 0.8598 - weighted_accuracy: 0.8348 - val_loss: 0.3602 - val_acc: 0.8315 - val_weighted_accuracy: 0.7947
Epoch 3/500
280483/280483 [==============================] - 44s 158us/step - loss: 0.2875 - acc: 0.8717 - weighted_accuracy: 0.8478 - val_loss: 0.3437 - val_acc: 0.8376 - val_weighted_accuracy: 0.8091
Epoch 4/500
280483/280483 [==============================] - 43s 155us/step - loss: 0.2743 - acc: 0.8790 - weighted_accuracy: 0.8563 - val_loss: 0.3459 - val_acc: 0.8384 - val_weighted_accuracy: 0.8032
Epoch 5/500
280483/280483 [==============================] - 43s 155us/step - loss: 0.2647 - acc: 0.8836 - weighted_accuracy: 0.8616 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 47s 167us/step - loss: 0.4673 - acc: 0.8071 - weighted_accuracy: 0.7697 - val_loss: 0.3567 - val_acc: 0.8324 - val_weighted_accuracy: 0.8107
Epoch 2/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.3247 - acc: 0.8509 - weighted_accuracy: 0.8237 - val_loss: 0.3415 - val_acc: 0.8414 - val_weighted_accuracy: 0.8261
Epoch 3/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.3034 - acc: 0.8632 - weighted_accuracy: 0.8379 - val_loss: 0.3278 - val_acc: 0.8514 - val_weighted_accuracy: 0.8343
Epoch 4/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2873 - acc: 0.8717 - weighted_accuracy: 0.8479 - val_loss: 0.3130 - val_acc: 0.8581 - val_weighted_accuracy: 0.8311
Epoch 5/500
280483/280483 [==============================] - 42s 150us/step - loss: 0.2745 - acc: 0.8790 - weighted_accuracy: 0.8563 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 47s 167us/step - loss: 0.4163 - acc: 0.8130 - weighted_accuracy: 0.7781 - val_loss: 0.3218 - val_acc: 0.8487 - val_weighted_accuracy: 0.8196
Epoch 2/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.3188 - acc: 0.8543 - weighted_accuracy: 0.8291 - val_loss: 0.3016 - val_acc: 0.8581 - val_weighted_accuracy: 0.8295
Epoch 3/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2964 - acc: 0.8671 - weighted_accuracy: 0.8436 - val_loss: 0.2949 - val_acc: 0.8651 - val_weighted_accuracy: 0.8322
Epoch 4/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2830 - acc: 0.8743 - weighted_accuracy: 0.8520 - val_loss: 0.2926 - val_acc: 0.8680 - val_weighted_accuracy: 0.8440
Epoch 5/500
280483/280483 [==============================] - 42s 149us/step - loss: 0.2713 - acc: 0.8805 - weighted_accuracy: 0.8588 - val_lo

In [101]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

0.8440403623403105

# MultiWindow CNN

In [29]:
def get_multiwindow_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    meta_features_input = Input(shape=(36,), name='mata-features')
    
    
    embedding = Embedding(nb_words, embedding_dim,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.2)(q1_embed)
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.2)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    
    q1_encoded = th(q1_embed,)    
    q2_encoded = th(q2_embed,)
    
    q1_in = q1_encoded
    q2_in = q2_encoded
    
    nb_filters = 64
    
    for i in range(1, 5):
        tanh_conv = Conv1D(nb_filters, i, padding='same', activation='tanh')
        sigm_conv = Conv1D(nb_filters, i, padding='same', activation='sigmoid')
        res_conv = Conv1D(nb_filters, i, padding='same', activation='relu')
        drop = Dropout(0.1)
        
        q1_t = tanh_conv(q1_in)
        q1_s = sigm_conv(q1_in)
        q1_x = Multiply()([q1_t, q1_s])
        
        res_q1 = res_conv(q1_x)
        res_q1 = drop(res_q1)
        q1_encoded = Concatenate()([q1_encoded, q1_x])
        
        q2_t = tanh_conv(q2_in)
        q2_s = sigm_conv(q2_in)       
        q2_x = Multiply()([q2_t, q2_s])
        
        res_q2 = res_conv(q2_x)
        res_q2 = drop(res_q2)
        q2_encoded = Concatenate()([q2_encoded, q2_x])

    # Align after align
    q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)
    q1_encoded = Concatenate()([q1_encoded, q2_aligned,])
    q2_encoded = Concatenate()([q2_encoded, q1_aligned,])
    
    #q1_encoded = Concatenate()([q1_embed, q1_compares])
    #q2_encoded = Concatenate()([q2_embed, q2_compares])

    #gru = Bidirectional(CuDNNGRU(72, return_sequences=True,))
    
    #q1_agg = gru(q1_compares)
    #q2_agg = gru(q2_compares)
   
    attn = AttentionWeightedAverage()
    q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn,])
    q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn,])    
    
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.2)(h_all)
    #h_all = Highway(activation='relu')(h_all)
    #h_all = Dropout(0.2)(h_all)
    #h_all = Highway(activation='relu')(h_all)    
    #h_all = Dense(256, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, meta_features_input], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

## TenCent

In [39]:
fold_count = 8
#embedding_matrix = sgns_bigram_matrix
embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 200

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordTC-Gated4GWindows-NoMeta-3P-NoClassWeighted-3Layers-NoEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    model_class_weights = None
    
    def _agent_get_model():
        return get_multiwindow_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
        em_train_features=em_train_features,                                             
        scale_sample_weight=None, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=20)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordTC-Gated4GWindows-NoMeta-3P-NoEM-NoClassWeighted-3Layers-NoEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    score = numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 30, 200)      20000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_7 (SpatialDro (None, 30, 200)      0           embedding_4[0][0]                
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 59s 209us/step - loss: 0.3514 - acc: 0.8383 - weighted_accuracy: 0.8094 - val_loss: 0.3269 - val_acc: 0.8484 - val_weighted_accuracy: 0.8350
Epoch 2/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2988 - acc: 0.8671 - weighted_accuracy: 0.8440 - val_loss: 0.3462 - val_acc: 0.8432 - val_weighted_accuracy: 0.8349
Epoch 3/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2819 - acc: 0.8764 - weighted_accuracy: 0.8551 - val_loss: 0.3123 - val_acc: 0.8574 - val_weighted_accuracy: 0.8442
Epoch 4/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2688 - acc: 0.8834 - weighted_accuracy: 0.8630 - val_loss: 0.3111 - val_acc: 0.8620 - val_weighted_accuracy: 0.8485
Epoch 5/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2588 - acc: 0.8881 - weighted_accuracy: 0.8690 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 58s 207us/step - loss: 0.3508 - acc: 0.8385 - weighted_accuracy: 0.8098 - val_loss: 0.2987 - val_acc: 0.8647 - val_weighted_accuracy: 0.8443
Epoch 2/500
280483/280483 [==============================] - 57s 202us/step - loss: 0.3007 - acc: 0.8661 - weighted_accuracy: 0.8430 - val_loss: 0.2966 - val_acc: 0.8647 - val_weighted_accuracy: 0.8530
Epoch 3/500
280483/280483 [==============================] - 57s 202us/step - loss: 0.2847 - acc: 0.8746 - weighted_accuracy: 0.8530 - val_loss: 0.3054 - val_acc: 0.8618 - val_weighted_accuracy: 0.8494
Epoch 4/500
280483/280483 [==============================] - 57s 202us/step - loss: 0.2710 - acc: 0.8816 - weighted_accuracy: 0.8611 - val_loss: 0.2830 - val_acc: 0.8746 - val_weighted_accuracy: 0.8555
Epoch 5/500
280483/280483 [==============================] - 57s 202us/step - loss: 0.2619 - acc: 0.8863 - weighted_accuracy: 0.8672 - val_lo

...

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 59s 209us/step - loss: 0.3471 - acc: 0.8408 - weighted_accuracy: 0.8118 - val_loss: 0.3247 - val_acc: 0.8522 - val_weighted_accuracy: 0.8360
Epoch 2/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2990 - acc: 0.8665 - weighted_accuracy: 0.8432 - val_loss: 0.3089 - val_acc: 0.8591 - val_weighted_accuracy: 0.8394
Epoch 3/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2813 - acc: 0.8767 - weighted_accuracy: 0.8552 - val_loss: 0.3216 - val_acc: 0.8568 - val_weighted_accuracy: 0.8476oss: 0.2818 - acc: 0.8765 - weighte
Epoch 4/500
280483/280483 [==============================] - 57s 202us/step - loss: 0.2694 - acc: 0.8825 - weighted_accuracy: 0.8623 - val_loss: 0.3130 - val_acc: 0.8612 - val_weighted_accuracy: 0.8466
Epoch 5/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2595 - acc: 0.8879 -

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 59s 210us/step - loss: 0.3465 - acc: 0.8418 - weighted_accuracy: 0.8128 - val_loss: 0.3348 - val_acc: 0.8443 - val_weighted_accuracy: 0.81624 - acc: 0.8408 - weighted_a - ETA: 2s - loss: 0.3477 - ac
Epoch 2/500
280483/280483 [==============================] - 57s 203us/step - loss: 0.2980 - acc: 0.8672 - weighted_accuracy: 0.8436 - val_loss: 0.3166 - val_acc: 0.8540 - val_weighted_accuracy: 0.8325
Epoch 3/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2812 - acc: 0.8762 - weighted_accuracy: 0.8540 - val_loss: 0.3189 - val_acc: 0.8538 - val_weighted_accuracy: 0.8404
Epoch 4/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2675 - acc: 0.8839 - weighted_accuracy: 0.8634 - val_loss: 0.3131 - val_acc: 0.8579 - val_weighted_accuracy: 0.8398
Epoch 5/500
280483/280483 [==============================] - 57s 204us/step - loss:

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 60s 213us/step - loss: 0.3455 - acc: 0.8407 - weighted_accuracy: 0.8123 - val_loss: 0.3608 - val_acc: 0.8304 - val_weighted_accuracy: 0.8093
Epoch 2/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2951 - acc: 0.8689 - weighted_accuracy: 0.8463 - val_loss: 0.3604 - val_acc: 0.8395 - val_weighted_accuracy: 0.8038
Epoch 3/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2784 - acc: 0.8779 - weighted_accuracy: 0.8564 - val_loss: 0.3456 - val_acc: 0.8436 - val_weighted_accuracy: 0.8173
Epoch 4/500
280483/280483 [==============================] - 57s 204us/step - loss: 0.2650 - acc: 0.8849 - weighted_accuracy: 0.8647 - val_loss: 0.3388 - val_acc: 0.8462 - val_weighted_accuracy: 0.8199
Epoch 5/500
280483/280483 [==============================] - 57s 205us/step - loss: 0.2552 - acc: 0.8900 - weighted_accuracy: 0.8703 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.3497 - acc: 0.8395 - weighted_accuracy: 0.8101 - val_loss: 0.3351 - val_acc: 0.8448 - val_weighted_accuracy: 0.8296
Epoch 2/500
280483/280483 [==============================] - 58s 205us/step - loss: 0.2997 - acc: 0.8667 - weighted_accuracy: 0.8430 - val_loss: 0.3211 - val_acc: 0.8550 - val_weighted_accuracy: 0.8365
Epoch 3/500
280483/280483 [==============================] - 58s 205us/step - loss: 0.2816 - acc: 0.8762 - weighted_accuracy: 0.8545 - val_loss: 0.3077 - val_acc: 0.8628 - val_weighted_accuracy: 0.8454
Epoch 4/500
280483/280483 [==============================] - 58s 206us/step - loss: 0.2703 - acc: 0.8825 - weighted_accuracy: 0.8620 - val_loss: 0.3068 - val_acc: 0.8635 - val_weighted_accuracy: 0.8504
Epoch 5/500
280483/280483 [==============================] - 58s 207us/step - loss: 0.2598 - acc: 0.8878 - weighted_accuracy: 0.8684 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.3489 - acc: 0.8397 - weighted_accuracy: 0.8112 - val_loss: 0.3184 - val_acc: 0.8518 - val_weighted_accuracy: 0.8224
Epoch 2/500
280483/280483 [==============================] - 58s 205us/step - loss: 0.2990 - acc: 0.8668 - weighted_accuracy: 0.8440 - val_loss: 0.2959 - val_acc: 0.8671 - val_weighted_accuracy: 0.8396
Epoch 3/500
280483/280483 [==============================] - 58s 206us/step - loss: 0.2824 - acc: 0.8757 - weighted_accuracy: 0.8550 - val_loss: 0.3164 - val_acc: 0.8567 - val_weighted_accuracy: 0.8435
Epoch 4/500
280483/280483 [==============================] - 58s 206us/step - loss: 0.2701 - acc: 0.8829 - weighted_accuracy: 0.8629 - val_loss: 0.2905 - val_acc: 0.8718 - val_weighted_accuracy: 0.8413
Epoch 5/500
280483/280483 [==============================] - 58s 205us/step - loss: 0.2604 - acc: 0.8870 - weighted_accuracy: 0.8681 - val_lo

In [207]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

## SGNS

In [30]:
fold_count = 8
embedding_matrix = sgns_bigram_matrix
#embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 300

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "SGNS-Gated4GWindows-NoMeta-3P-NoClassWeighted-3Layers-NoEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    model_class_weights = None
    
    def _agent_get_model():
        return get_multiwindow_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
        em_train_features=em_train_features,                                             
        scale_sample_weight=None, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=20)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "SGNS-Gated4GWindows-NoMeta-3P-NoEM-NoClassWeighted-3Layers-NoEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    score = numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 300)      30000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 30, 300)      0           embedding_1[0][0]                
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
 34688/280483 [==>...........................] - ETA: 1:50 - loss: 0.4297 - acc: 0.7992 - weighted_accuracy: 0.7622

KeyboardInterrupt: 

In [ ]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

# ESIM

In [20]:
from keras import regularizers

def get_ESIM(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")
    
    embedding = Embedding(nb_words, embedding_dim,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    batch_norm = BatchNormalization(axis=-1)
    q1_embed = batch_norm(q1_embed,)
    q2_embed = batch_norm(q2_embed,)  
    
    aggreation_gru = Bidirectional(CuDNNLSTM(100, return_sequences=True))
 
    q1_seq = aggreation_gru(q1_embed)
    q2_seq = aggreation_gru(q2_embed)
        
    q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
    
    q1_vec = Concatenate()([q1_seq, q2_aligned, substract(q1_seq, q2_aligned), Multiply()([q1_seq, q2_aligned])])
    q2_vec = Concatenate()([q2_seq, q1_aligned, substract(q2_seq, q1_aligned), Multiply()([q2_seq, q1_aligned])])
    
    compare_gru = Bidirectional(CuDNNLSTM(100, return_sequences=True))
    
    q1_rep = compare_gru(q1_vec)
    q2_rep = compare_gru(q2_vec)
    
    q1_rep = apply_multiple(q1_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    h_all = Concatenate()([q1_rep, q2_rep])
    h_all = BatchNormalization()(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.5)(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.5)(h_all)
   
    out_ = Dense(3, activation='softmax')(h_all)
    
    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

## TenCent

In [35]:
fold_count = 8
#embedding_matrix = sgns_bigram_matrix
embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 200

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordTC-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    model_class_weights = None
    
    def _agent_get_model():
        return get_ESIM(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE, lr=4e-4)
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=64,
        em_train_features=em_train_features,                                             
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=7)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordTC-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


KeyboardInterrupt: 

In [ ]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

## SGNS

In [22]:
fold_count = 8
embedding_matrix = sgns_bigram_matrix
#embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 300

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    model_class_weights = None
    
    def _agent_get_model():
        return get_ESIM(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=64,
        em_train_features=em_train_features,                                             
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=10)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 30, 300)      30000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_17 (SpatialDr (None, 30, 300)      0           embedding_9[0][0]           

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 119s 424us/step - loss: 0.3928 - acc: 0.8238 - weighted_accuracy: 0.7919 - val_loss: 0.3708 - val_acc: 0.8245 - val_weighted_accuracy: 0.8155
Epoch 2/500
280483/280483 [==============================] - 116s 414us/step - loss: 0.3164 - acc: 0.8583 - weighted_accuracy: 0.8324 - val_loss: 0.4123 - val_acc: 0.8235 - val_weighted_accuracy: 0.8118
Epoch 3/500
280483/280483 [==============================] - 115s 408us/step - loss: 0.2920 - acc: 0.8715 - weighted_accuracy: 0.8477 - val_loss: 0.3304 - val_acc: 0.8539 - val_weighted_accuracy: 0.8352
Epoch 4/500
280483/280483 [==============================] - 115s 411us/step - loss: 0.2753 - acc: 0.8801 - weighted_accuracy: 0.8579 - val_loss: 0.3330 - val_acc: 0.8419 - val_weighted_accuracy: 0.8295
Epoch 5/500
280483/280483 [==============================] - 115s 408us/step - loss: 0.2623 - acc: 0.8875 - weighted_accuracy: 0.8660 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 119s 424us/step - loss: 0.3949 - acc: 0.8231 - weighted_accuracy: 0.7916 - val_loss: 0.3097 - val_acc: 0.8590 - val_weighted_accuracy: 0.8401
Epoch 2/500
280483/280483 [==============================] - 115s 412us/step - loss: 0.3161 - acc: 0.8591 - weighted_accuracy: 0.8339 - val_loss: 0.3122 - val_acc: 0.8568 - val_weighted_accuracy: 0.8433
Epoch 3/500
280483/280483 [==============================] - 116s 413us/step - loss: 0.2936 - acc: 0.8705 - weighted_accuracy: 0.8467 - val_loss: 0.3171 - val_acc: 0.8563 - val_weighted_accuracy: 0.8429
Epoch 4/500
280483/280483 [==============================] - 116s 414us/step - loss: 0.2770 - acc: 0.8801 - weighted_accuracy: 0.8584 - val_loss: 0.3003 - val_acc: 0.8668 - val_weighted_accuracy: 0.8562
Epoch 5/500
280483/280483 [==============================] - 116s 413us/step - loss: 0.2622 - acc: 0.8873 - weighted_accuracy: 0.8667 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 121s 432us/step - loss: 0.3926 - acc: 0.8247 - weighted_accuracy: 0.7911 - val_loss: 2.6884 - val_acc: 0.3052 - val_weighted_accuracy: 0.3045
Epoch 2/500
280483/280483 [==============================] - 115s 412us/step - loss: 0.3129 - acc: 0.8601 - weighted_accuracy: 0.8345 - val_loss: 1.8006 - val_acc: 0.4267 - val_weighted_accuracy: 0.4219
Epoch 3/500
280483/280483 [==============================] - 116s 413us/step - loss: 0.2898 - acc: 0.8735 - weighted_accuracy: 0.8494 - val_loss: 0.3093 - val_acc: 0.8559 - val_weighted_accuracy: 0.8409
Epoch 4/500
280483/280483 [==============================] - 116s 414us/step - loss: 0.2715 - acc: 0.8824 - weighted_accuracy: 0.8598 - val_loss: 0.3204 - val_acc: 0.8537 - val_weighted_accuracy: 0.8461
Epoch 5/500
280483/280483 [==============================] - 115s 410us/step - loss: 0.2575 - acc: 0.8898 - weighted_accuracy: 0.8684 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 121s 431us/step - loss: 0.3924 - acc: 0.8238 - weighted_accuracy: 0.7913 - val_loss: 0.5139 - val_acc: 0.7738 - val_weighted_accuracy: 0.7705
Epoch 2/500
280483/280483 [==============================] - 115s 409us/step - loss: 0.3129 - acc: 0.8602 - weighted_accuracy: 0.8345 - val_loss: 0.3234 - val_acc: 0.8577 - val_weighted_accuracy: 0.8411
Epoch 3/500
280483/280483 [==============================] - 116s 415us/step - loss: 0.2889 - acc: 0.8729 - weighted_accuracy: 0.8493 - val_loss: 0.3111 - val_acc: 0.8606 - val_weighted_accuracy: 0.8468
Epoch 4/500
280483/280483 [==============================] - 118s 419us/step - loss: 0.2704 - acc: 0.8826 - weighted_accuracy: 0.8603 - val_loss: 0.3096 - val_acc: 0.8603 - val_weighted_accuracy: 0.8413
Epoch 5/500
280483/280483 [==============================] - 117s 418us/step - loss: 0.2563 - acc: 0.8900 - weighted_accuracy: 0.8688 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 120s 428us/step - loss: 0.3915 - acc: 0.8242 - weighted_accuracy: 0.7920 - val_loss: 0.3494 - val_acc: 0.8364 - val_weighted_accuracy: 0.8208
Epoch 2/500
280483/280483 [==============================] - 115s 410us/step - loss: 0.3120 - acc: 0.8608 - weighted_accuracy: 0.8349 - val_loss: 0.3393 - val_acc: 0.8403 - val_weighted_accuracy: 0.8172
Epoch 3/500
280483/280483 [==============================] - 117s 417us/step - loss: 0.2903 - acc: 0.8729 - weighted_accuracy: 0.8491 - val_loss: 0.3297 - val_acc: 0.8491 - val_weighted_accuracy: 0.8337
Epoch 4/500
280483/280483 [==============================] - 116s 413us/step - loss: 0.2734 - acc: 0.8816 - weighted_accuracy: 0.8593 - val_loss: 0.3192 - val_acc: 0.8530 - val_weighted_accuracy: 0.8356
Epoch 5/500
280483/280483 [==============================] - 117s 417us/step - loss: 0.2584 - acc: 0.8889 - weighted_accuracy: 0.8676 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 123s 439us/step - loss: 0.3913 - acc: 0.8241 - weighted_accuracy: 0.7935 - val_loss: 0.4094 - val_acc: 0.8082 - val_weighted_accuracy: 0.7689
Epoch 2/500
280483/280483 [==============================] - 119s 425us/step - loss: 0.3112 - acc: 0.8614 - weighted_accuracy: 0.8361 - val_loss: 0.3666 - val_acc: 0.8288 - val_weighted_accuracy: 0.7979
Epoch 3/500
280483/280483 [==============================] - 118s 420us/step - loss: 0.2880 - acc: 0.8736 - weighted_accuracy: 0.8504 - val_loss: 0.3455 - val_acc: 0.8404 - val_weighted_accuracy: 0.8158
Epoch 4/500
280483/280483 [==============================] - 117s 417us/step - loss: 0.2694 - acc: 0.8832 - weighted_accuracy: 0.8616 - val_loss: 0.3461 - val_acc: 0.8390 - val_weighted_accuracy: 0.8216
Epoch 5/500
280483/280483 [==============================] - 119s 423us/step - loss: 0.2549 - acc: 0.8908 - weighted_accuracy: 0.8705 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 122s 437us/step - loss: 0.3922 - acc: 0.8239 - weighted_accuracy: 0.7927 - val_loss: 1.6386 - val_acc: 0.4497 - val_weighted_accuracy: 0.4375
Epoch 2/500
280483/280483 [==============================] - 118s 421us/step - loss: 0.3126 - acc: 0.8602 - weighted_accuracy: 0.8345 - val_loss: 0.5407 - val_acc: 0.7430 - val_weighted_accuracy: 0.7086
Epoch 3/500
280483/280483 [==============================] - 119s 423us/step - loss: 0.2889 - acc: 0.8729 - weighted_accuracy: 0.8495 - val_loss: 0.6132 - val_acc: 0.7105 - val_weighted_accuracy: 0.6907
Epoch 4/500
280483/280483 [==============================] - 118s 422us/step - loss: 0.2708 - acc: 0.8828 - weighted_accuracy: 0.8608 - val_loss: 1.4922 - val_acc: 0.3217 - val_weighted_accuracy: 0.3224
Epoch 5/500
280483/280483 [==============================] - 117s 417us/step - loss: 0.2560 - acc: 0.8899 - weighted_accuracy: 0.8689 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 123s 438us/step - loss: 0.3968 - acc: 0.8229 - weighted_accuracy: 0.7917 - val_loss: 2.2637 - val_acc: 0.2987 - val_weighted_accuracy: 0.2956
Epoch 2/500
280483/280483 [==============================] - 118s 421us/step - loss: 0.3186 - acc: 0.8573 - weighted_accuracy: 0.8317 - val_loss: 0.3164 - val_acc: 0.8554 - val_weighted_accuracy: 0.8379
Epoch 3/500
280483/280483 [==============================] - 117s 417us/step - loss: 0.2947 - acc: 0.8701 - weighted_accuracy: 0.8468 - val_loss: 0.2962 - val_acc: 0.8660 - val_weighted_accuracy: 0.8457
Epoch 4/500
280483/280483 [==============================] - 119s 425us/step - loss: 0.2771 - acc: 0.8792 - weighted_accuracy: 0.8573 - val_loss: 0.3068 - val_acc: 0.8639 - val_weighted_accuracy: 0.8467
Epoch 5/500
280483/280483 [==============================] - 118s 419us/step - loss: 0.2625 - acc: 0.8867 - weighted_accuracy: 0.8662 - v

In [ ]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)

# DenseRNN

In [22]:
from keras import regularizers

def get_darnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')

    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")
    
    embedding = Embedding(nb_words, embedding_dim,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    q1_embed = Dropout(0.1)(th(q1_embed,))
    q2_embed = Dropout(0.1)(th(q2_embed,))    
    
    rnns = [Bidirectional(CuDNNGRU(42, return_sequences=True)) for i in range(3)]
    
    q1_res = []
    q2_res = []
    
    
    for idx, rnn in enumerate(rnns):
        q1_seq = rnn(q1_embed)
        q1_seq = Dropout(0.15)(q1_seq)
        q2_seq = rnn(q2_embed)
        q2_seq = Dropout(0.15)(q2_seq)
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        
        q1_res.append(q2_aligned)
        q1_res.append(q1_seq)
        
        q2_res.append(q1_aligned)
        q2_res.append(q2_seq)
        
        q1_embed = Concatenate()([q1_embed, q1_seq, q2_aligned,])
        q2_embed = Concatenate()([q2_embed, q2_seq, q1_aligned,])            
        
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))

    q1_res = Concatenate()(q1_res)
    q2_res = Concatenate()(q2_res)
    
    attn = AttentionWeightedAverage()
    q1_rep = apply_multiple(q1_embed, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])
    q2_rep = apply_multiple(q2_embed, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])   
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.35)(h_all)
    h_all = Dense(300, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipvalue=1.5), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

## TenCent

In [43]:
fold_count = 8
#embedding_matrix = sgns_bigram_matrix
embedding_matrix = tencent_ai_matrix
EMBEDDING_DIM = 200

for i in range(1, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "WordTC-DenseRNN-NoMeta-3P-NoEM-NoClassWeighted-3Layers"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    model_class_weights = None
    
    def _agent_get_model():
        return get_darnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=1024,
        em_train_features=em_train_features,                                             
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=20)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "WordTC-DenseRNN-NoMeta-3P-NoEM-NoClassWeighted-3Layers"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": tests_1_ems,
                                       "second_exact_match": tests_2_ems,
                                      }, batch_size=128, verbose=1)

        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub.to_csv(submit_path, index=False)
    break

Work on model 1


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 30)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 30, 200)      20000000    first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_27 (SpatialDr (None, 30, 200)      0           embedding_16[0][0]               
__________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 51s 180us/step - loss: 0.5372 - acc: 0.7264 - weighted_accuracy: 0.6798 - val_loss: 0.4122 - val_acc: 0.7976 - val_weighted_accuracy: 0.7592
Epoch 2/500
280483/280483 [==============================] - 47s 168us/step - loss: 0.4110 - acc: 0.7989 - weighted_accuracy: 0.7594 - val_loss: 0.3881 - val_acc: 0.8121 - val_weighted_accuracy: 0.7842
Epoch 3/500
280483/280483 [==============================] - 46s 166us/step - loss: 0.3650 - acc: 0.8287 - weighted_accuracy: 0.7965 - val_loss: 0.3501 - val_acc: 0.8332 - val_weighted_accuracy: 0.8121
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3318 - acc: 0.8478 - weighted_accuracy: 0.8201 - val_loss: 0.3254 - val_acc: 0.8484 - val_weighted_accuracy: 0.8284
Epoch 5/500
280483/280483 [==============================] - 47s 169us/step - loss: 0.3111 - acc: 0.8586 - weighted_accuracy: 0.8329 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 50s 177us/step - loss: 0.5668 - acc: 0.7142 - weighted_accuracy: 0.6672 - val_loss: 0.4206 - val_acc: 0.8020 - val_weighted_accuracy: 0.7625
Epoch 2/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4226 - acc: 0.7905 - weighted_accuracy: 0.7518 - val_loss: 0.3753 - val_acc: 0.8244 - val_weighted_accuracy: 0.7901
Epoch 3/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3749 - acc: 0.8220 - weighted_accuracy: 0.7888 - val_loss: 0.3316 - val_acc: 0.8459 - val_weighted_accuracy: 0.8196
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3383 - acc: 0.8428 - weighted_accuracy: 0.8148 - val_loss: 0.3271 - val_acc: 0.8447 - val_weighted_accuracy: 0.8265
Epoch 5/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3171 - acc: 0.8548 - weighted_accuracy: 0.8285 - val_lo

...

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 50s 179us/step - loss: 0.5360 - acc: 0.7295 - weighted_accuracy: 0.6833 - val_loss: 0.4385 - val_acc: 0.7811 - val_weighted_accuracy: 0.7396
Epoch 2/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4076 - acc: 0.7999 - weighted_accuracy: 0.7616 - val_loss: 0.3833 - val_acc: 0.8163 - val_weighted_accuracy: 0.7821
Epoch 3/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3550 - acc: 0.8333 - weighted_accuracy: 0.8018 - val_loss: 0.3545 - val_acc: 0.8310 - val_weighted_accuracy: 0.8173
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3230 - acc: 0.8507 - weighted_accuracy: 0.8234 - val_loss: 0.3345 - val_acc: 0.8427 - val_weighted_accuracy: 0.8277
Epoch 5/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3074 - acc: 0.8600 - weighted_accuracy: 0.8342 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 50s 180us/step - loss: 0.5615 - acc: 0.7190 - weighted_accuracy: 0.6717 - val_loss: 0.4451 - val_acc: 0.7775 - val_weighted_accuracy: 0.7379
Epoch 2/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4184 - acc: 0.7930 - weighted_accuracy: 0.7534 - val_loss: 0.4020 - val_acc: 0.8072 - val_weighted_accuracy: 0.7755
Epoch 3/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3674 - acc: 0.8274 - weighted_accuracy: 0.7937 - val_loss: 0.3595 - val_acc: 0.8310 - val_weighted_accuracy: 0.8027
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3299 - acc: 0.8489 - weighted_accuracy: 0.8205 - val_loss: 0.3620 - val_acc: 0.8265 - val_weighted_accuracy: 0.8086
Epoch 5/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3097 - acc: 0.8594 - weighted_accuracy: 0.8331 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 51s 180us/step - loss: 0.5302 - acc: 0.7299 - weighted_accuracy: 0.6868 - val_loss: 0.4722 - val_acc: 0.7598 - val_weighted_accuracy: 0.7111
Epoch 2/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4025 - acc: 0.8037 - weighted_accuracy: 0.7654 - val_loss: 0.4140 - val_acc: 0.7941 - val_weighted_accuracy: 0.7590
Epoch 3/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3488 - acc: 0.8365 - weighted_accuracy: 0.8052 - val_loss: 0.3708 - val_acc: 0.8201 - val_weighted_accuracy: 0.7864
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3190 - acc: 0.8540 - weighted_accuracy: 0.8264 - val_loss: 0.3623 - val_acc: 0.8236 - val_weighted_accuracy: 0.8070
Epoch 5/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3033 - acc: 0.8618 - weighted_accuracy: 0.8360 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 51s 182us/step - loss: 0.5957 - acc: 0.7127 - weighted_accuracy: 0.6674 - val_loss: 0.4569 - val_acc: 0.7743 - val_weighted_accuracy: 0.7325
Epoch 2/500
280483/280483 [==============================] - 46s 166us/step - loss: 0.4275 - acc: 0.7872 - weighted_accuracy: 0.7459 - val_loss: 0.4048 - val_acc: 0.8096 - val_weighted_accuracy: 0.7663
Epoch 3/500
280483/280483 [==============================] - 47s 166us/step - loss: 0.3798 - acc: 0.8189 - weighted_accuracy: 0.7836 - val_loss: 0.3697 - val_acc: 0.8256 - val_weighted_accuracy: 0.8036
Epoch 4/500
280483/280483 [==============================] - 47s 166us/step - loss: 0.3441 - acc: 0.8400 - weighted_accuracy: 0.8104 - val_loss: 0.3495 - val_acc: 0.8353 - val_weighted_accuracy: 0.8206
Epoch 5/500
280483/280483 [==============================] - 47s 166us/step - loss: 0.3206 - acc: 0.8528 - weighted_accuracy: 0.8257 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 51s 182us/step - loss: 0.5573 - acc: 0.7179 - weighted_accuracy: 0.6720 - val_loss: 0.4177 - val_acc: 0.7988 - val_weighted_accuracy: 0.7583
Epoch 2/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.4209 - acc: 0.7901 - weighted_accuracy: 0.7513 - val_loss: 0.3687 - val_acc: 0.8236 - val_weighted_accuracy: 0.7867
Epoch 3/500
280483/280483 [==============================] - 46s 166us/step - loss: 0.3686 - acc: 0.8246 - weighted_accuracy: 0.7934 - val_loss: 0.3496 - val_acc: 0.8341 - val_weighted_accuracy: 0.8076
Epoch 4/500
280483/280483 [==============================] - 46s 165us/step - loss: 0.3337 - acc: 0.8455 - weighted_accuracy: 0.8182 - val_loss: 0.3395 - val_acc: 0.8370 - val_weighted_accuracy: 0.8164
Epoch 5/500
280483/280483 [==============================] - 46s 166us/step - loss: 0.3132 - acc: 0.8573 - weighted_accuracy: 0.8314 - val_lo

In [ ]:
numpy_weighted_accuracy(labels, oofs[['unrelated', 'agreed', 'disagreed']].values)